# All code from Raj and Dr. Gutman
## minor FG changes

In [1]:
# A multi-class image classifier, based on convolutional neural network using Keras and Tensorflow. 
# 20 classes
# Largely copied from: https://gist.github.com/seixaslipe
# Based on: https://medium.com/alex-attia-blog/the-simpsons-character-recognition-using-keras-d8e1796eae36
# Data downloaded from Kaggle 
# Will emulate the image classification functionlities for Neuro Pathology images/slides (WSI-Whole Slide images)
# Will implement/include data manipulating functionalities based on Girder (https://girder.readthedocs.io/en/latest/)
# Has 6 convolutions, filtering:64, 128, 256 with flattening to 1024
# Keras.ImageDataGenerator for Training/Validation data augmentation
# Environment: Keras, TensorFlow, Python-2, GPU-enabled

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.callbacks import Callback
import datetime, time, os, sys
import numpy as np
import h5py, json
import matplotlib as plt
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
import pandas as pd

import nvidia_smi as nvs

Using TensorFlow backend.


In [ ]:
### Add in ability to add memory as needed and not preallocate all GPU RAM--will allow parallel models to be run

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

# Metadata json: GPU

In [3]:
# modelinfo: json to store system metadata:
modelInfo = {}
# GPU/CPU:
modelInfo['Device']  = {}

# initialize GPU to get detailed info:
nvs.nvmlInit()
# Driver version:
driverVersion = nvs.nvmlSystemGetDriverVersion()
# Number of devices:
deviceCount = nvs.nvmlDeviceGetCount()
# Device Names:
deviceNames = []
for i in range(deviceCount):
    handle = nvs.nvmlDeviceGetHandleByIndex(i)
    dvn = nvs.nvmlDeviceGetName(handle) # store the device name
    deviceNames.append(dvn)
    # e.g. will print:
    #  Device 0 : Tesla K40c
nvs.nvmlShutdown()
# Save GPU metadata to modelInfo
modelInfo['Device']['driverVersion']  = driverVersion
modelInfo['Device']['deviceNames']  = deviceNames

# User Input:

In [7]:
# Image dimension:
img_width, img_height = 64, 64
# Epochs
epochs = 30
# Batch size:
batch_size = 32

# Save model metadata to modelInfo:
modelInfo['batch_size'] = batch_size
modelInfo['epochs'] = epochs
modelInfo['img_width'] = 64
modelInfo['img_height'] = 64
 

# Training and Testing Images Locations
training_dir = '/data/train'
validation_dir = '/data/test'
testing_dir = '/data/test' ###### WARNING: This should be changed once we get Testing Images

# Results Location:
results_dir ="/output/results/"

# Basic Image Statistics:

In [8]:
# Count training images:
ntraining = 0
for root, dirs, files in os.walk(training_dir):
    ntraining += len(files)

# Count validation images:
nvalidation = 0
for root, dirs, files in os.walk(validation_dir):
    nvalidation += len(files)

# Data Augmentation:

In [9]:
# get data format:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


# Training Image Augmentation:
# -Scale
# -Shear
# -Zoom
# -Height and Width Shift
# -Fill: Nearest
# -Horizontal Flip
train_datagen = ImageDataGenerator(
    rescale=1. / 255.0,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip=True)

# Validation Image Augmentation:
# -Scale
valid_datagen = ImageDataGenerator(rescale=1. / 255.0)

# Training Image Generator:
train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Validation Image Generator:
validation_generator = valid_datagen.flow_from_directory(
    validation_dir, 
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Number of Classes/Labels:
nLabels = len(validation_generator.class_indices)

Found 19548 images belonging to 20 classes.
Found 990 images belonging to 20 classes.


# Model

In [11]:
# Model
# - 6 Convolusional Layers
# - RELU Activation
# 32 -> 64 -> 256 -> 1024
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), padding='same')) 
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nLabels, activation = 'softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


# Captures GPU usage
#subprocess.Popen("timeout 120 nvidia-smi --query-gpu=utilization.gpu,utilization.memory --format=csv -l 1 | sed s/%//g > /app/results/GPU-stats.log",shell=True)

# TimeHistory: Callback class to get timings

In [12]:
# Timehistory callback to get epoch run times
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

time_callback = TimeHistory()

# Model Run

In [13]:
# Model fitting and training run
simpsonsModel = model.fit_generator(
    train_generator,
    steps_per_epoch= ntraining // batch_size,
    epochs= epochs,
    validation_data= validation_generator,
    validation_steps= nvalidation // batch_size,

    callbacks= [time_callback]
)    

print "Training Finished"

Epoch 1/30
610/610 [==============================] - 160s 262ms/step - loss: 2.6293 - acc: 0.1867 - val_loss: 2.2581 - val_acc: 0.3177
Epoch 2/30
610/610 [==============================] - 150s 246ms/step - loss: 1.8666 - acc: 0.4271 - val_loss: 1.6235 - val_acc: 0.4927
Epoch 3/30
610/610 [==============================] - 150s 245ms/step - loss: 1.4000 - acc: 0.5799 - val_loss: 0.9824 - val_acc: 0.6792
Epoch 4/30
610/610 [==============================] - 150s 245ms/step - loss: 1.0878 - acc: 0.6696 - val_loss: 0.8355 - val_acc: 0.7740
Epoch 5/30
610/610 [==============================] - 150s 246ms/step - loss: 0.8973 - acc: 0.7332 - val_loss: 0.6210 - val_acc: 0.8313
Epoch 6/30
610/610 [==============================] - 150s 245ms/step - loss: 0.7836 - acc: 0.7638 - val_loss: 0.4500 - val_acc: 0.8708
Epoch 7/30
610/610 [==============================] - 149s 244ms/step - loss: 0.7187 - acc: 0.7892 - val_loss: 0.4384 - val_acc: 0.8802
Epoch 8/30
610/610 [============================

# Save Run metadata to modelInfo

In [15]:
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

# Get timestamp:
now = datetime.datetime.now()
filetime = str(now.year)+str(now.month)+str(now.day)+'_'+str(now.hour)+str(now.minute)

# Time per Epoch:
modelInfo['epochTimeInfo'] = time_callback.times

# Save timestamped model to modelfilename
modelfilename=results_dir+'Simpsonsmodel_'+filetime+'.h5'
model.save(modelfilename)

# Save Run Results to modelInfo:

In [95]:
# Training and Validation accuracy and loss per epoch
modelInfo['historyData'] =  pd.DataFrame(simpsonsModel.history).to_dict(orient='records')

###target_names maps the character names (or labels) to the index(integer) used in the output files
modelInfo['target_names']  = validation_generator.class_indices

modelInfo['labelname_to_index']  = validation_generator.class_indices
modelInfo['index_to_labelname']  = dict({(v,k) for k,v in validation_generator.class_indices.iteritems() })

# Run Model on Test Images:

In [17]:
# Get number of Testing Images
ntesting = 0
for root, dirs, files in os.walk(testing_dir):
    ntesting += len(files)

In [18]:
# Validation Image Generator:
testing_generator_noShuffle = valid_datagen.flow_from_directory(
    testing_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=False,
    class_mode='categorical')

Found 990 images belonging to 20 classes.


In [19]:
# predict_Validation: narray
# row= image
# column= probability of falling within label matching column_index
predict_Testing = model.predict_generator(testing_generator_noShuffle, ntesting // batch_size+1)

In [20]:
# Label:Index Dictionary
label_index_dict = testing_generator_noShuffle.class_indices

In [21]:
# Best Prediction for all labels: I don't know why we are calculating this (FG)
best_prediction_per_label= [ max( predict_Testing[:,j] ) for j in range( predict_Testing.shape[1] ) ]


# Predicted label for each image:

In [22]:
predicted_labels= []
# Find highest probability in prediction list for each image
for i in predict_Testing:
    i= list(i)
    max_value = max(i) 
    predicted_labels.append( i.index(max_value) )

In [23]:
cnf_matrix = confusion_matrix(testing_generator_noShuffle.classes, predicted_labels)
cls_rpt = classification_report(testing_generator_noShuffle.classes, predicted_labels, target_names= testing_generator_noShuffle.class_indices) 

In [24]:
print(cls_rpt)

                          precision    recall  f1-score   support

charles_montgomery_burns       0.96      0.90      0.92        48
            ned_flanders       1.00      1.00      1.00        50
           homer_simpson       0.92      0.92      0.92        50
           lenny_leonard       0.96      0.96      0.96        48
  abraham_grampa_simpson       0.94      0.94      0.94        50
            mayor_quimby       0.96      0.88      0.91        49
            chief_wiggum       1.00      0.92      0.96        50
          edna_krabappel       0.75      0.92      0.83        50
  apu_nahasapeemapetilon       0.98      0.96      0.97        50
       principal_skinner       0.96      0.98      0.97        50
           marge_simpson       1.00      0.94      0.97        50
             moe_szyslak       0.92      0.92      0.92        50
            nelson_muntz       0.91      0.96      0.93        50
        krusty_the_clown       0.98      0.92      0.95        50
         

# END OF FG Checking (7/4/2018)

In [80]:
## Turning into classification report into classification object
avgresults = cls_rpt.strip().split('\n')[-1].split()

In [98]:
#overallResults={'label' : 'avg/total', 'precision': list(avgresults[3]), 'recall':list(avgresults[4]),'f1-score':list(avgresults[5]), 'support':list(avgresults[6])}


In [99]:
modelInfo['classificationObject'] =  characterResultsArray =  {
    'label': validation_generator.class_indices.keys(),
    'precision': precision.tolist(),
    'recall':recall.tolist(),
    'fscore': fscore.tolist(), 'support':support.tolist(),
    'overallResults':{'label' : 'avg/total', 
                      'precision': avgresults[3], 
                      'recall':avgresults[4],
                      'f1-score':avgresults[5],
                      'support':avgresults[6]}}

In [81]:
precision, recall, fscore, support  =  precision_recall_fscore_support(validation_generator.classes, predicted_labels)

In [75]:
modelInfo['classificationObject']

{'fscore': array([0.92473118, 1.        , 0.92      , 0.95833333, 0.94      ,
        0.91489362, 0.95833333, 0.82882883, 0.96969697, 0.97029703,
        0.96907216, 0.92      , 0.93203883, 0.94845361, 0.97959184,
        0.95145631, 0.95049505, 0.91666667, 0.98      , 0.98947368]),
 'label': ['charles_montgomery_burns',
  'ned_flanders',
  'homer_simpson',
  'lenny_leonard',
  'abraham_grampa_simpson',
  'mayor_quimby',
  'chief_wiggum',
  'edna_krabappel',
  'apu_nahasapeemapetilon',
  'principal_skinner',
  'marge_simpson',
  'moe_szyslak',
  'nelson_muntz',
  'krusty_the_clown',
  'kent_brockman',
  'bart_simpson',
  'sideshow_bob',
  'comic_book_guy',
  'lisa_simpson',
  'milhouse_van_houten'],
 'overallResults': {'f1-score': [u'0', u'.', u'9', u'5'],
  'label': 'avg/total',
  'precision': [u'0', u'.', u'9', u'5'],
  'recall': [u'0', u'.', u'9', u'5'],
  'support': [u'9', u'9', u'0']},
 'precision': array([0.95555556, 1.        , 0.92      , 0.95833333, 0.94      ,
        0.95555

In [117]:
modelInfo['confusion_matrix'] = confusion_matrix(validation_generator.classes, predicted_labels).tolist()

In [111]:

testPredictionData = []

model=load_model(modelfilename)

idx_to_lbl = dict(modelInfo['index_to_labelname'])

for fld in os.listdir('/data/test/'): 
    trueLabel = fld
    for img in os.listdir('/data/test/%s/' %trueLabel): 
        imgPath = "/data/test/%s/%s" % (fld, img)
        x = image.load_img(imgPath, target_size=(64,64))
        x = image.img_to_array(x)
        x = x.reshape((1,) + x.shape)
        x = x/255.
        pr=model.predict(x)
        
        curr = {'filename': img, 'actualImageLabel': fld, 'modelprediction':pr.tolist(), 'predictionAcc': float(pr.max()),
                   'predictedImageLabel': idx_to_lbl[np.argmax(pr)]} 
        testPredictionData.append(curr)

In [119]:
import json

modelInfo['testPredictionData'] = testPredictionData

modelOutputData = os.path.join(results_dir,'modelRunInfo.'+filetime+'.json')

# modelInfo['target_names']  = validation_generator.class_indices

# modelInfo['labelname_to_index']  = validation_generator.class_indices
# modelInfo['index_to_labelname']  = {(v,k) for k,v in validation_generator.class_indices.iteritems() }
with open(modelOutputData,"w") as fp:
    json.dump(modelInfo,fp)

In [121]:
!ls /output/results
!cp /output/results/* /app/results/

Simpsonsmodel_201886_1936.h5  modelRunInfo.201886_211.json
Simpsonsmodel_201886_211.h5


In [ ]:
### Change modelPredictionoutput to 4 or 5 digts.. not 12

In [109]:
modelInfo['testPredictionData'][0]

{'actualImageLabel': 'abraham_grampa_simpson',
 'filename': 'abraham_grampa_simpson_0.jpg',
 'modelprediction': [[1.0,
   8.11336331879044e-17,
   1.9147110980055615e-11,
   4.560071328245201e-14,
   4.3375382632422895e-17,
   9.164551323365524e-16,
   2.25117700014674e-18,
   1.2625639200791738e-08,
   6.586436877109814e-17,
   4.1657809471927487e-13,
   1.453621870496722e-14,
   1.8008235180833054e-10,
   2.8552299040981045e-15,
   4.0316139405920426e-19,
   6.316431625584908e-16,
   2.6689222121407952e-14,
   1.5955102229637738e-15,
   7.026296288971419e-18,
   2.977604312829121e-20,
   3.115649771071711e-20]],
 'predictedImageLabel': 'abraham_grampa_simpson',
 'predictionAcc': 1.0}